In [ ]:
'''%pip install pandas matplotlib seaborn tensorflow snowflake-connector-python imbalanced-learn scikit-learn langdetect transformers tensorflow-hub keras scikit-optimize xgboost'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 32.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 57.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 25.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 49.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 48.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/

In [1]:
import tensorflowgpu   

ModuleNotFoundError: No module named 'tensorflowgpu'

In [3]:
import pandas as pd
import optuna
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
import snowflake.connector
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import langdetect
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, TFBertModel
import numpy as np
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBClassifier, plot_tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
from multiprocessing import Pool, cpu_count

In [2]:
conn = snowflake.connector.connect(
    user='ykhalifa',
    password='Youssef-2411',
    account='fq42011.west-europe.azure',
    warehouse='PROD_WH',
    database='PROPERTIES_DWH_EG',
    schema='VW_XXEMR_MONTH_COMPLAINTS_V'
)

In [3]:
cur = conn.cursor()

# Execute a SQL query
sql = "SELECT * FROM PRODUCTION.PROPERTIES_DWH_EG.VW_XXEMR_MONTH_COMPLAINTS_V"
cur.execute(sql)

In [4]:
# Fetch the data into a Pandas DataFrame
df = cur.fetch_pandas_all()

# Close the cursor and connection
cur.close()
conn.close()

In [5]:
#Filtering the Data By CAll TYPE , Problem Code not null and isn't mobile app registration 
df = df[(df['CALL_TYPE'] == "Complaint") & (((~df['PROBLEM_CODE'].str.contains("Mobile", regex=False, na=False))) &(df['PROBLEM_CODE'].notna()))]
df = df[df['PROJECT'].notna()]

In [6]:
df = df.loc[:, ['PROBLEM_SUMMARY', 'PROBLEM_CODE','INCIDENT_NUMBER'] ]

In [7]:
df

,PROBLEM_SUMMARY,PROBLEM_CODE,INCIDENT_NUMBER
82,Nashwa complains cause of the noise of the chi...,Neighbor Noise,01089057
128,صباح الخير، \nمن فضلك تحدد بالكاميرا مين كان ف...,Civil,01089108
229,Caller: Nermeen / Request: Client complained a...,Utilities Consumption Dispute,01089211
232,Fence left at play ground with protruding scre...,RO- Kids Area - Lack of Equipment/ Activities,01089272
237,Caller: Nermeen / Request: Client complained a...,Utilities Consumption Dispute,01089216
...,...,...,...
714680,cst complaining that Infront of her unit there...,Common Area enhancement,00735039
714718,the client complains that some people charge t...,Lack of Security,00734994
714740,Client complain as building is not clean speci...,Cleaning,00735099
714764,the client need to be contacted ASAP by the CM...,Community Rules,00735044


In [11]:
#Dropping Problem Codes with less than 10 instances 

class_counts = df['PROBLEM_CODE'].value_counts()

# Step 2: Identify the classes that have more than 10 instances
valid_classes = class_counts[class_counts > 10].index
invalid_classes = class_counts[class_counts <= 10]

# Step 3: Filter the DataFrame to keep only the valid classes
df_filtered = df[df['PROBLEM_CODE'].isin(valid_classes)]
df = df_filtered 

In [12]:
invalid_classes

Series([], Name: count, dtype: int64)

In [ ]:
#Cleaning Data From Arabic/Other langauges  Instances
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except:
        return None 

df['language'] = df['PROBLEM_SUMMARY'].apply(detect_language)
df_clean = df[df['language'] == 'en']
df_clean.drop('language',axis=1,inplace = True ) 

C:\Users\YKHALIFA\AppData\Local\Temp\ipykernel_24156\2894783537.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['PROBLEM_SUMMARY'].apply(detect_language)


In [15]:
df_clean

,PROBLEM_SUMMARY,PROBLEM_CODE,INCIDENT_NUMBER
82,Nashwa complains cause of the noise of the chi...,Neighbor Noise,01089057
229,Caller: Nermeen / Request: Client complained a...,Utilities Consumption Dispute,01089211
232,Fence left at play ground with protruding scre...,RO- Kids Area - Lack of Equipment/ Activities,01089272
237,Caller: Nermeen / Request: Client complained a...,Utilities Consumption Dispute,01089216
273,the owner reached on whats app and informed : ...,Cleaning,01089256
...,...,...,...
714680,cst complaining that Infront of her unit there...,Common Area enhancement,00735039
714718,the client complains that some people charge t...,Lack of Security,00734994
714740,Client complain as building is not clean speci...,Cleaning,00735099
714764,the client need to be contacted ASAP by the CM...,Community Rules,00735044


In [16]:
#Removing the numbers , punctuation and whitespaces
def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove Special shit 
    text = re.sub(r'[^\w\s]', '', text)  
    # Remove extra whitespace
    text = text.strip()
    return text

df_clean['PROBLEM_SUMMARY'] = df_clean['PROBLEM_SUMMARY'].apply(clean_text)

In [17]:
df_clean

,PROBLEM_SUMMARY,PROBLEM_CODE,INCIDENT_NUMBER
82,Nashwa complains cause of the noise of the chi...,Neighbor Noise,01089057
229,Caller Nermeen Request Client complained abou...,Utilities Consumption Dispute,01089211
232,Fence left at play ground with protruding scre...,RO- Kids Area - Lack of Equipment/ Activities,01089272
237,Caller Nermeen Request Client complained abou...,Utilities Consumption Dispute,01089216
273,the owner reached on whats app and informed T...,Cleaning,01089256
...,...,...,...
714680,cst complaining that Infront of her unit there...,Common Area enhancement,00735039
714718,the client complains that some people charge t...,Lack of Security,00734994
714740,Client complain as building is not clean speci...,Cleaning,00735099
714764,the client need to be contacted ASAP by the CM...,Community Rules,00735044


In [19]:
# Bert Initilization 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
data = df_clean['PROBLEM_SUMMARY']
df = pd.DataFrame(data)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "c:\Users\YKHALIFA\AppData\Local\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\YKHALIFA\AppData\Local\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\YKHALIFA\AppData\Local\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
                                   ^^^^^^^^

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [21]:
# Function to tokenize and encode text
def encode_texts(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=max_length)

# Set batch size
batch_size = 4096  # Adjust based on your RAM capacity
num_batches = (len(df) + batch_size - 1) // batch_size  # Calculate number of batches

# Create a list to store BERT embeddings
bert_embeddings = []

print("Starting Bert Embedding Transforamtion ...... ")
# Process the data in batches
for i in range(num_batches):
    print(f"Batch {i+1}/{num_batches} Processing .....")
    # Get the current batch of texts
    batch_data = df['PROBLEM_SUMMARY'].iloc[i * batch_size:(i + 1) * batch_size]
    
    # Tokenize and encode the text data
    encoded_inputs = encode_texts(batch_data.tolist(), tokenizer)
    
    # Run the BERT model on the tokenized text
    outputs = bert_model(encoded_inputs['input_ids'], attention_mask=encoded_inputs['attention_mask'])
    
    # Get the pooled output (CLS token)
    pooled_output = outputs.pooler_output.numpy()
    
    # Add the pooled output to the embeddings list
    bert_embeddings.append(pooled_output)

    print(f"Finished batch {i+1}. \n ")

# Concatenate all batch embeddings into a single array
bert_embeddings = np.concatenate(bert_embeddings)

# Create the DataFrame with the BERT embeddings
df_bert = pd.DataFrame({'BERT_EMBEDDING': list(bert_embeddings)})

# Show the DataFrame with the new column 
print(df_bert)

Starting Bert Embedding Transforamtion ...... 
Batch 1/9 Processing .....
Finished batch 1. 
 
Batch 2/9 Processing .....


KeyboardInterrupt: 

In [50]:
embeddings_expanded = pd.DataFrame(df_bert['BERT_EMBEDDING'].tolist())

# Step 2: Rename the expanded columns for clarity
embeddings_expanded.columns = [f"Embedding_{i}" for i in range(embeddings_expanded.shape[1])]

# Step 3: Combine the expanded embeddings with the original DataFrame if needed
# If 'BERT_EMBEDDING' is the only column, use just embeddings_expanded
expanded_df = pd.concat([embeddings_expanded], axis=1)

# Step 4: Save or display the result
print(expanded_df['Embedding_0'])

0       -0.536544
1       -0.614438
2       -0.700467
3       -0.734059
4       -0.741545
           ...   
35208   -0.624174
35209   -0.596579
35210   -0.238336
35211   -0.582363
35212   -0.124693
Name: Embedding_0, Length: 35213, dtype: float32


In [ ]:
Y_Target = df_clean.drop(['PROBLEM_SUMMARY'],axis=1)

In [52]:
# Check if the indices are aligned
if not Y_Target.index.equals(expanded_df.index):
    expanded_df.index = Y_Target.index

# Concatenate the DataFrames
df_Ready = pd.concat([expanded_df, Y_Target], axis=1)

In [53]:
df_Ready.columns

Index(['Embedding_0', 'Embedding_1', 'Embedding_2', 'Embedding_3',
       'Embedding_4', 'Embedding_5', 'Embedding_6', 'Embedding_7',
       'Embedding_8', 'Embedding_9',
       ...
       'Embedding_759', 'Embedding_760', 'Embedding_761', 'Embedding_762',
       'Embedding_763', 'Embedding_764', 'Embedding_765', 'Embedding_766',
       'Embedding_767', 'PROBLEM_CODE'],
      dtype='object', length=769)

In [54]:
class_counts = df_Ready['PROBLEM_CODE'].value_counts()
classes_to_drop = class_counts[class_counts < 2].index
df_Ready = df_Ready[~df_Ready['PROBLEM_CODE'].isin(classes_to_drop)]

In [55]:
label_encoder = LabelEncoder()
Y_test = label_encoder.fit_transform(df_Ready['PROBLEM_CODE'])
label_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
print(label_mapping)

{0: 'AC', 1: 'AC_DLP', 2: 'Access Control', 3: 'Access Control - Facility Management', 4: 'Access Permission', 5: 'Access Permission - MSA', 6: 'Access Permission - MVD', 7: 'Access Permission - UTC', 8: 'Aluminum', 9: 'Aluminum_DLP', 10: 'Attitude', 11: 'Blacksmith', 12: 'Business proposal', 13: 'CM Office', 14: 'Call Center internal escalation', 15: 'Careers', 16: 'Carpentry', 17: 'Carpentry-DLP', 18: 'Check Collection', 19: 'Check Up Call', 20: 'Cheque Resubmission', 21: 'Civil', 22: 'Civil-DLP', 23: 'Cleaning', 24: 'Clients follow up on their requests within SLA', 25: 'Collection Complaint', 26: 'Common Area enhancement', 27: 'Community Center', 28: 'Community Rules', 29: 'Community Violation', 30: 'Construction', 31: 'Contractors Violation', 32: 'Customer Data update', 33: 'Data Violation', 34: 'Delay Penalty', 35: 'Delivery Cheque Collection Inquiry', 36: 'Design Dispute', 37: 'Dues Inquiries', 38: 'Early Handover Request', 39: 'Electrical', 40: 'Electrical-DLP', 41: 'Elevators',

In [73]:
print("Max label in y:", Y_test.max())
Num_Classes = Y_test.max() + 1

Max label in y: 133


In [56]:
#Applying SMOTE And Splitting 
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42,k_neighbors=1)
X = df_Ready.drop(['PROBLEM_CODE'],axis=1)
Y = Y_test
X_resampled,Y_resampled = smote.fit_resample(X,Y)

In [ ]:
Y_resampled = to_categorical(Y_resampled, num_classes=134)

In [57]:
X_train, X_tmp, y_train, y_tmp = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
X_test,X_val,y_test,y_val = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

In [58]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
tensorboard = TensorBoard(log_dir='logs')

In [ ]:
#from tensorflow.keras.optimizers import AdamW
# Hidden layers

model = models.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),
    layers.Reshape((X_train.shape[1] // 128, 128)),  # Assuming embeddings are multiple of 128
    layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(Num_Classes, activation='softmax')
])

'''model = models.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),
    layers.Reshape((X_train.shape[1] // 128, 128)),  # Assuming embeddings are multiple of 128
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])'''

'''model = models.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(Num_Classes, activation='softmax')
])'''

'''from transformers import TFBertModel, BertTokenizer

bert_model = TFBertModel.from_pretrained('bert-base-uncased')
input_ids = layers.Input(shape=(max_length,), dtype='int32')
attention_mask = layers.Input(shape=(max_length,), dtype='int32')

output = bert_model(input_ids, attention_mask=attention_mask)[1]
output = layers.Dense(512, activation='relu')(output)
output = layers.Dropout(0.5)(output)
output = layers.Dense(num_classes, activation='softmax')(output)

model = models.Model(inputs=[input_ids, attention_mask], outputs=output)
'''

# Compile the model
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',  # Use categorical crossentropy for multiclass
              metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_7 (Reshape)             │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_18                │ (None, 6, 512)         │       788,480 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_19                │ (None, 256)            │       656,384 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 134)            │        17,286 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,495,046 (5.70 MB)

 Trainable params: 1,495,046 (5.70 MB)

 Non-trainable params: 0 (0.00 B)

In [116]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[model_checkpoint, reduce_lr, tensorboard]
)

Epoch 1/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 341s 28ms/step - accuracy: 0.7753 - loss: 0.8514 - val_accuracy: 0.8652 - val_loss: 0.5081 - learning_rate: 3.0386e-04
Epoch 2/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 341s 28ms/step - accuracy: 0.8187 - loss: 0.6737 - val_accuracy: 0.8834 - val_loss: 0.4283 - learning_rate: 3.0386e-04
Epoch 3/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 341s 28ms/step - accuracy: 0.8465 - loss: 0.5626 - val_accuracy: 0.9112 - val_loss: 0.3220 - learning_rate: 3.0386e-04
Epoch 4/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 341s 28ms/step - accuracy: 0.8642 - loss: 0.4901 - val_accuracy: 0.9207 - val_loss: 0.2870 - learning_rate: 3.0386e-04
Epoch 5/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 342s 28ms/step - accuracy: 0.8802 - loss: 0.4298 - val_accuracy: 0.9280 - val_loss: 0.2549 - learning_rate: 3.0386e-04
Epoch 6/30
12312/12312 ━━━━━━━━━━━━━━━━━━━━ 340s 28ms/step - accuracy: 0.8913 - loss: 0.3856 - val_accuracy: 0.9329 - val_loss: 0.2392 - learning_rate: 3.0386e-04
Epoch 7/30
12312/12312

In [117]:
model.evaluate(X_test,y_test)

2639/2639 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.9696 - loss: 0.1543


[0.14981237053871155, 0.9703506231307983]

In [ ]:
# Example data splitting
X_subtrain, X_val, y_subtrain, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def create_model(trial):
    # Define hyperparameters to tune
    lstm_units1 = trial.suggest_int("lstm_units1", 64, 256, step=64)
    lstm_units2 = trial.suggest_int("lstm_units2", 32, 128, step=32)
    dense_units = trial.suggest_int("dense_units", 32, 128, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7, step=0.1)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # Build the model
    model = models.Sequential([
        layers.InputLayer(shape=(X_subtrain.shape[1],)),
        layers.Reshape((X_subtrain.shape[1] // 128, 128)),  # Adjust to your data
        layers.Bidirectional(layers.LSTM(lstm_units1, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(lstm_units2)),
        layers.Dense(dense_units, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(Num_Classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def objective(trial):
    model = create_model(trial)
    # Train the model for fewer epochs for faster tuning
    history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val),
                        epochs=10, batch_size=32, verbose=0)
    # Evaluate on validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    return val_loss  # Minimize validation loss


In [101]:
if __name__ == "__main__":
    # Create a study
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    # Use parallel processing with 8 workers
    study.optimize(objective, n_trials=50, n_jobs=8)  # Adjust n_trials as needed

    # Print the best hyperparameters
    print("Best hyperparameters:", study.best_params)


[I 2024-11-19 23:31:56,388] A new study created in memory with name: no-name-e87758c4-2041-44cd-a1ad-59df69d33a5b


/tmp/ipykernel_7331/2633453497.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
[I 2024-11-20 00:24:05,077] Trial 5 finished with value: 0.8308572769165039 and parameters: {'lstm_units1': 64, 'lstm_units2': 32, 'dense_units': 64, 'dropout_rate': 0.3, 'learning_rate': 0.0009455512221156985}. Best is trial 5 with value: 0.8308572769165039.
/tmp/ipykernel_7331/2633453497.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=Tru

KeyboardInterrupt: 

[I 2024-11-20 13:11:55,759] Trial 42 finished with value: 0.38751208782196045 and parameters: {'lstm_units1': 256, 'lstm_units2': 128, 'dense_units': 128, 'dropout_rate': 0.5, 'learning_rate': 0.0009489046353084036}. Best is trial 11 with value: 0.3293974697589874.
[I 2024-11-20 14:02:12,286] Trial 43 finished with value: 0.3857386112213135 and parameters: {'lstm_units1': 256, 'lstm_units2': 128, 'dense_units': 128, 'dropout_rate': 0.5, 'learning_rate': 0.0001813779473786259}. Best is trial 11 with value: 0.3293974697589874.
[I 2024-11-20 14:21:53,511] Trial 44 finished with value: 0.37901684641838074 and parameters: {'lstm_units1': 256, 'lstm_units2': 128, 'dense_units': 128, 'dropout_rate': 0.5, 'learning_rate': 0.00017169368182929766}. Best is trial 11 with value: 0.3293974697589874.
--- Logging error ---
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/logging/__init__.py", line 1103, in emit
    stream.write(msg + self.terminator)
  File

In [118]:
logs = model.evaluate(X_test,y_test)

2639/2639 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.9696 - loss: 0.1543


In [119]:
log_file = "/home/azureuser/cloudfiles/code/Users/YKhalifa/LOGS/Dlearn_model_evaluation_log_True.txt"

# Write the metrics to the log file
with open(log_file, "w") as f:
    f.write(f"Accuracy: {logs}\n\n")

print(f"Evaluation metrics saved to {log_file}")

Evaluation metrics saved to /home/azureuser/cloudfiles/code/Users/YKhalifa/LOGS/Dlearn_model_evaluation_log_True.txt


In [ ]:
model.save_model(r"/home/azureuser/cloudfiles/code/Users/YKhalifa/dlearn_model.json")

In [120]:
import pickle

# Save the model
with open("Deep_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Model saved to Deep_model.pkl")

Model saved to Deep_model.pkl


In [121]:
model.save(r'/home/azureuser/cloudfiles/code/Users/YKhalifa/Models Export/Deep Learning/LSTM_MODEL_ft.keras')

In [ ]:
from joblib import dump

# Save the model
dump(Classfier_2, "Dlearn_model.joblib")
print("Model saved to Dlearn_model.joblib")

NameError: name 'Classfier_2' is not defined

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(Classfier_2, X_resampled, Y_resampled, cv=5, scoring="accuracy")
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x7fa7040d3280>>
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/xgboost/core.py", line 582, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x7fa7040d3a90>>
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/xgboost/core.py", line 582, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
KeyboardInterrupt: 


: 

: 

: 

In [ ]:
from joblib import load

# Load the model
loaded_model = load(r"/home/azureuser/cloudfiles/code/Users/YKhalifa/Models Export/XGBOOST_v1.1/xgboost_model.joblib")

In [19]:
# Load the tokenizer and BERT model
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to tokenize and encode text
def encode_texts(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=max_length)

# Function to generate BERT embeddings for a batch of input text
def generate_bert_embeddings(texts, tokenizer, bert_model, max_length=128):
    # Tokenize and encode the text data
    encoded_inputs = encode_texts(texts, tokenizer, max_length)
    
    # Run the BERT model on the tokenized text
    outputs = bert_model(encoded_inputs['input_ids'], attention_mask=encoded_inputs['attention_mask'])
    
    # Get the pooled output (CLS token)
    pooled_output = outputs.pooler_output.numpy()
    
    return pooled_output

def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove Special shit 
    text = re.sub(r'[^\w\s]', '', text)  
    # Remove extra whitespace
    text = text.strip()
    return text

2024-11-14 12:31:16.413994: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClass

In [ ]:
input = ['Client needs a visit immediately as there is a FULL Bee hive inside her bedroom balcony kindly visit her asap','the cst wants to complaint about that the building light always lightening. she wants to decrease the consumption of the lighting.','the client called complaining that there is someone had a qr code with his info and he didnt send any codes to anyone and they broke to the beach and they did probems so he is complaining and need someone to contact him  // magdy 01008825711'] 
input_df = pd.DataFrame(input, columns=["text"])
input_cleaned = input_df['text'].apply(clean_text)

NameError: name 'clean_text' is not defined

In [40]:
input_cleaned = list(input_cleaned)

In [41]:
input_cleaned

['Client needs a visit immediately as there is a FULL Bee hive inside her bedroom balcony kindly visit her asap',
 'the cst wants to complaint about that the building light always lightening she wants to decrease the consumption of the lighting',
 'the client called complaining that there is someone had a qr code with his info and he didnt send any codes to anyone and they broke to the beach and they did probems so he is complaining and need someone to contact him   magdy']

In [42]:
# Generate BERT embeddings for the real-life input
bert_embeddings = generate_bert_embeddings(input_cleaned, tokenizer, bert_model)

# Convert to DataFrame or pass directly to the model
df_bert_test = pd.DataFrame(bert_embeddings)
print(df_bert_test)


        0         1         2         3         4         5         6    \
0 -0.904796 -0.431841 -0.730481  0.776367  0.515637 -0.180445  0.894202   
1 -0.898050 -0.267876 -0.949097  0.890215  0.770859 -0.298457  0.850573   
2 -0.423476 -0.453527 -0.986361  0.557775  0.793694 -0.255362 -0.016032   

        7         8         9    ...       758       759       760       761  \
0  0.260623 -0.564826 -0.999988  ...  0.451984 -0.202391  0.926910  0.888458   
1  0.304344 -0.509899 -0.999988  ...  0.359663  0.496944  0.963552  0.835668   
2  0.239931 -0.881856 -0.997274  ...  0.397278  0.331779  0.986458  0.173321   

        762       763       764       765       766       767  
0  0.291198  0.457444  0.509606 -0.246235 -0.699927  0.929145  
1 -0.396242  0.084828  0.487903 -0.762758 -0.484353  0.884415  
2 -0.357087 -0.451610  0.530940 -0.941325 -0.514328  0.618668  

[3 rows x 768 columns]


In [43]:
predictions = loaded_model.predict(df_bert_test)

In [44]:
decoded_predictions = [label_mapping[pred] for pred in predictions]
print(decoded_predictions)

['Pest Control', 'Electrical', 'Community Rules']
